In [5]:
%pip install vit_pytorch

     ---------------------------------------- 0.0/65.7 kB ? eta -:--:--
     ---------------------------------------- 65.7/65.7 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/119.7 kB ? eta -:--:--
   ---------------------------------------- 119.7/119.7 kB 7.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/43.2 kB ? eta -:--:--
   ---------------------------------------- 43.2/43.2 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dataloader import training_dataset, testing_dataset
import torch
from torch.utils.data import DataLoader
from vit_pytorch import ViT, SimpleViT
from torch import nn
from tqdm import tqdm
import os

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
training_data = DataLoader(training_dataset, batch_size=32, shuffle=True)
testing_data = DataLoader(testing_dataset, batch_size=32, shuffle=True)

In [4]:
vision_model = ViT(
    image_size=128 * 3,
    patch_size=32,
    num_classes=39,
    dim=512,
    depth=6,
    heads=16,
    mlp_dim=1024,
    dropout = 0.1,
    emb_dropout = 0.1
).to('cuda')

# vit_model = nn.Sequential(vision_model, nn.Sigmoid())

In [5]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(vision_model.parameters(), lr=1e-7)

In [6]:
state = {'loss': [], 'epochs': [], 'test_loss': []}
def train_loop(dataloader, testing_data, model, loss_fn, optimizer, epochs=10, save_path='./saves'):
    global state
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    all_exps = [int(elm.replace('exp_', '')) if elm != '.ipynb_checkpoints' else -1 for elm in os.listdir(save_path)]

    current_num = max(all_exps) if (len(all_exps) > 0) else 1
    save_path = save_path + '/exp_' + str(current_num + 1)
    os.mkdir(save_path)

    model.to(device)

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        loop = tqdm(dataloader, total=len(dataloader), leave=True)
        total_loss = 0

        for batch_idx, (data, targets) in enumerate(loop):
            data = data.to(device)
            targets = targets.to(device)

            # Forward pass
            scores = model(data)
            loss = loss_fn(scores, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Update progress bar
            loop.set_postfix(loss=loss.item())

        testing_loss = 0
        for test in tqdm(testing_data):
            X, y = test
            with torch.no_grad():
                model.eval()

                score = model(X)
                testing_loss =+ loss_fn(score, y)

                model.train()

        print(f"Epoch {epoch + 1} average loss: {total_loss / len(dataloader)} with testing loss of {testing_loss / len(testing_data)}")

        state['loss'].append(total_loss / len(dataloader))
        state['epochs'].append(epoch + 1)
        state['test_loss'].append((testing_loss / len(testing_data)).cpu())

        # Save the model after each epoch
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss / len(dataloader),
            'testing_loss': (testing_loss / len(testing_data)).cpu()
        }, f"{save_path}/model_epoch_{epoch + 1}.pth")

    print("Training complete!")

In [7]:
train_loop(training_data, testing_data, vision_model, loss_fn, optimizer, epochs=100)

Epoch 1/100


100%|██████████| 13/13 [00:13<00:00,  1.06s/it]


Epoch 1 average loss: 0.6990987602621317 with testing loss of 0.052406664937734604
Epoch 2/100


100%|██████████| 13/13 [00:12<00:00,  1.07it/s]


Epoch 2 average loss: 0.6859181514009833 with testing loss of 0.05120355263352394
Epoch 3/100


...

Epoch 98 average loss: 0.265776411164552 with testing loss of 0.017748035490512848
Epoch 99/100


100%|██████████| 13/13 [00:11<00:00,  1.10it/s]


Epoch 99 average loss: 0.2652573613449931 with testing loss of 0.020988788455724716
Epoch 100/100


100%|██████████| 13/13 [00:12<00:00,  1.04it/s]


Epoch 100 average loss: 0.26479940698482096 with testing loss of 0.016836050897836685
Training complete!


In [9]:
state.keys()

dict_keys(['loss', 'epochs', 'test_loss'])

In [13]:
plt.scatter()

[tensor(0.0524),
 tensor(0.0512),
 tensor(0.0510),
 tensor(0.0500),
 tensor(0.0509),
 tensor(0.0472),
 tensor(0.0469),
 tensor(0.0469),
 tensor(0.0446),
 tensor(0.0439),
 tensor(0.0438),
 tensor(0.0455),
 tensor(0.0415),
 tensor(0.0419),
 tensor(0.0395),
 tensor(0.0433),
 tensor(0.0422),
 tensor(0.0380),
 tensor(0.0374),
 tensor(0.0418),
 tensor(0.0372),
 tensor(0.0384),
 tensor(0.0354),
 tensor(0.0375),
 tensor(0.0356),
 tensor(0.0370),
 tensor(0.0337),
 tensor(0.0324),
 tensor(0.0329),
 tensor(0.0333),
 tensor(0.0308),
 tensor(0.0295),
 tensor(0.0303),
 tensor(0.0294),
 tensor(0.0329),
 tensor(0.0256),
 tensor(0.0320),
 tensor(0.0325),
 tensor(0.0275),
 tensor(0.0286),
 tensor(0.0241),
 tensor(0.0277),
 tensor(0.0279),
 tensor(0.0264),
 tensor(0.0270),
 tensor(0.0286),
 tensor(0.0240),
 tensor(0.0254),
 tensor(0.0254),
 tensor(0.0254),
 tensor(0.0253),
 tensor(0.0269),
 tensor(0.0229),
 tensor(0.0281),
 tensor(0.0247),
 tensor(0.0249),
 tensor(0.0247),
 tensor(0.0228),
 tensor(0.0225